# NIDLookUP (USE ilearn2.0)
*   程式簡介
> 校內目前並未提供任何可以查尋 NID 的方式，本次利用 ilearn 提供之簡易查詢功能（NID 查詢姓名），實作出一個可以暴破計算出欲查詢者 NID 的腳本（反推爆破 使姓名可以返查 NID）

*   主要查詢
> 目前查詢功能的實作已經可以因應各種模式，並透過不同的參數引入，可查詢更多的 NID 資料

# Import


In [0]:
from bs4 import BeautifulSoup
import requests, getpass, csv, re

# Set Session

In [0]:
s = requests.Session()

# Set login Data

In [0]:
username = 'D0618990'
password = 'yitsewuAB569854'

# username = input('請輸入學號:')
# password = getpass.getpass('請輸入密碼:')

# ilearn Tool

## 功能 00：Login ilearn

In [6]:
# Set login url
loginPage_url = 'https://ilearn2.fcu.edu.tw/login/index.php'

# Get the logintoken
loginPage = s.get(loginPage_url)
bs_loginPage = BeautifulSoup(loginPage.text, 'html.parser')

login_data = {
    'username': username,
    'password': password,
    'logintoken' : bs_loginPage.find_all(attrs={"name": "logintoken"})[0]['value']
}

# login
try:
    loginPage = s.post(loginPage_url, data = login_data)
    bs_loginPage = BeautifulSoup(loginPage.text, 'html.parser')

    # print(loginPage.text)
    print('%s 登入成功' %(bs_loginPage.find_all(attrs={"class":"userpicture defaultuserpic"})[0]['alt'][0:3]))

except:
    print('登入失敗')

巫逸哲 登入成功


## 功能 01：查詢所有授課老師
*   ilearn ID
*   課程名稱
*   老師名稱





In [0]:
# define course
class course():
    def __init__(self, name, ID):
        self.name = name
        self.ID = ID
        self.teacher = []
    def add_teachet(self, teacher):
        self.teacher.append(teacher)

# All_course_list
All_course = []

# Set url
url = 'https://ilearn2.fcu.edu.tw/'
page = s.get(url)
bs_page = BeautifulSoup(page.text, 'html.parser')

# 查詢課表
course_list = bs_loginPage.find_all(attrs={"class":"coc-mycurricular coursebox clearfix"})

# print(len(course_list))

for i in range(len(course_list)):
    try:
        course_name = course_list[i].find_all('a')[0]['title']
        
        course_id = course_list[i].find_all('a')[0]['href'][46:51]

        temp = course(course_name, course_id)

        try:
            course_teacher = course_list[i].find_all('span')
            for j in range(len(course_teacher)):
                if j == 0:
                    course_teacher_name = course_teacher[j].text[4:]
                    temp.add_teachet(course_teacher_name)
                else:
                    course_teacher_name = course_teacher[j].text[1:]
                    temp.add_teachet(course_teacher_name)
        except:
            print()

        All_course.append(temp)

    except:
        print('Error')

In [10]:
for i in All_course:
    print('ID：' + i.ID)
    print('URL：'+ 'https://ilearn2.fcu.edu.tw/user/index.php?id=' + i.ID)
    print('課程名稱：' + i.name)
    print('老師：' + str(i.teacher))
    print()

ID：70650
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70650
課程名稱：1081 程式設計(I)(資訊一乙)[1525]
老師：['劉明機']

ID：70651
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70651
課程名稱：1081 程式設計(II)(資訊一乙)[1526]
老師：['劉明機']

ID：70653
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70653
課程名稱：1081 離散數學(資訊二丁)[1566]
老師：['游景盛']

ID：71240
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=71240
課程名稱：1081 THIRD-YEAR ENGLISH(資訊三乙)[1575]
老師：['林芷瑩']

ID：70630
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70630
課程名稱：1081 作業系統(一)(資訊三乙)[1576]
老師：['林志敏']

ID：70613
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70613
課程名稱：1081 資料庫系統(電腦學程資訊三)[2304]
老師：['林明言']

ID：70611
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70611
課程名稱：1081 社群網路APP開發與應用(軟工學程資訊三)[2308]
老師：['劉明機']

ID：70602
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70602
課程名稱：1081 程式語言(軟工學程資訊三)[2306]
老師：['林峰正']

ID：70663
URL：https://ilearn2.fcu.edu.tw/user/index.php?id=70663
課程名稱：1081 物件導向軟體工程(資訊跨域學程資訊三)[2349]
老師：['江庭輝']

ID：68843
URL：https://i

## 功能02：查詢 NID

In [0]:
def SearchNID(Course_ID, status, name):
    # Set NID & StudentMode
    NID = status
    StudentMode = True

    # Set URL
    url = ('https://ilearn2.fcu.edu.tw/user/index.php?id=' + str(Course_ID))
    page = s.get(url)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    # Get contextid
    contextid = bs_page.find_all('body')[0]['class'][11][8:]

    # Get perpage（課程參加人數）
    perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

    # Fix URL
    url = ('https://ilearn2.fcu.edu.tw/user/index.php?contextid='+ contextid +'&id=' + Course_ID + '&perpage=' + perpage)

    # Check StudentMode
    page = s.get(url)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    try:
        if bs_page.find_all(attrs={"value": "加入用戶到此課程"})[0]['value'] == '加入用戶到此課程':
            StudentMode = False
            MemberList = bs_page.find_all('tbody')[0]
        for i in range(int(perpage)):
            Member_name = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r"+ str(i) +"_c1"})[0].find('img')['alt'][0:3]
            Member_NID = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(i) + "_c2"})[0].text
            Member_status = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(i) + "_c3"})[0].text[14:-15]
            
            if Member_status == '助理教師':
                Member_status = '助教'
            
            if Member_name == name:
                return Member_NID
        
            if name == 'ALL':
                print ('%3s %-8s %s' %(Member_name, Member_NID, Member_status))
    except:
        if name == 'ALL' and StudentMode == True:
            return '權限不足'

    # 檢查是否有此人
    data = {
        'unified-filters[]': name,
        'unified-filter-submitted': '1'
    }

    page = s.post(url, data = data)
    bs_page = BeautifulSoup(page.text, 'html.parser')

    try:
        check = bs_page.find_all(attrs={"id": "user-index-participants-" + Course_ID + "_r0_c0"})[0].find('img')['alt'][0:3]
        perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]
        (name == check and perpage == '1') == True
    except:
        return ('查無此人：')

    searchMode = 0

    # 搜尋模式：老師
    if status == 'T':
        searchMode = 6
    elif status == 'D':
        searchMode = 8
    
    while len(NID) != searchMode:
        for i in range(0, 10):
            # print(NID)
            data = {
                'unified-filters[]': NID + str(i),
                'unified-filter-submitted': '1'
            }

            page = s.post(url, data = data)
            bs_page = BeautifulSoup(page.text, 'html.parser')
            
            
            perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

            try:
                MemberList = bs_page.find_all('tbody')[0]
                
                Still_in_list = False

                for j in range(int(perpage)):
                    Member_name = MemberList.find_all(attrs={"id":"user-index-participants-" + Course_ID + "_r" + str(j) +"_c0"})[0].find('img')['alt'][0:3]
                    Member_status = MemberList.find_all(attrs={"id":"user-index-participants-"+ Course_ID + "_r" + str(j) + "_c1"})[0].text
                    if name == Member_name:
                        Still_in_list = True
                        break

                if Still_in_list == True:
                    NID += str(i)
                else:
                    continue
            except:
                continue
    return NID

## 功能03：顯示所有老師證號 與 輸出 CSV

In [16]:
with open('outputCSV-NID.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['ID', '課程', '老師姓名 證號'])

    for i in All_course:
        print('ID：' + i.ID)
        print('課程：' + i.name)
        print('老師：', end='')

        if i.teacher == None:
            print('\n')
        
        tempstr = ''

        for j in i.teacher:
            tempNID = SearchNID(i.ID, 'T', j)
            
            print(j, tempNID ,end=' ')

            tempstr = tempstr + j + ' ' + tempNID + ' '
        
        writer.writerow([i.ID, i.name, tempstr])

        print('\n')

ID：70650
課程：1081 程式設計(I)(資訊一乙)[1525]
老師：劉明機 T06052 

ID：70651
課程：1081 程式設計(II)(資訊一乙)[1526]
老師：劉明機 T06052 

ID：70653
課程：1081 離散數學(資訊二丁)[1566]
老師：游景盛 T96210 

ID：71240
課程：1081 THIRD-YEAR ENGLISH(資訊三乙)[1575]
老師：林芷瑩 T01119 

ID：70630
課程：1081 作業系統(一)(資訊三乙)[1576]
老師：林志敏 T81218 

ID：70613
課程：1081 資料庫系統(電腦學程資訊三)[2304]
老師：林明言 T91158 

ID：70611
課程：1081 社群網路APP開發與應用(軟工學程資訊三)[2308]
老師：劉明機 T06052 

ID：70602
課程：1081 程式語言(軟工學程資訊三)[2306]
老師：林峰正 T00287 

ID：70663
課程：1081 物件導向軟體工程(資訊跨域學程資訊三)[2349]
老師：江庭輝 T08030 

ID：68843
課程：1081 物聯網概論(物聯網學程)[3236]
老師：陳家豪 T04063 

ID：69323
課程：1081 服務學習(綜合班)[3586]
老師：

ID：68423
課程：1074-微積分(二)(暑修班)
老師：陳長城 T75060 

ID：68189
課程：1072 翻轉逢甲 —逢甲資源平台建置(通統合(M))[(日)01-02][4613]
老師：魏國瑞 T06024 魏國瑞 T06024 

ID：68191
課程：逢甲大學\資電學院\資訊系學生資訊站
老師：陳雅真 T82142 薛念林 T91125 黃鎮南 T92195 吳邦華 T95118 粘巧鈴 T96252 

ID：68143
課程：1072跑跑無人車─AI於無人車之創新應用[00028]
老師：林昱成 T02182 李靜儀 T02214 吳沛儒 T03063 陳楷婷 T06285 謝佩君 T94167 

ID：22006
課程：E課程總表
老師：周雪惠 T94145 鄭雪娥 T94111 



## 功能04：輸出 CVS 檔案

In [0]:
with open('output.csv', 'w', newline='') as csvfile:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvfile)

    writer.writerow(['ID', '課程', '老師姓名 證號'])
    
    for i in All_course:
        

        for j in i.teacher:
            tempstr = tempstr + str(SearchNID(i.ID, 'T', j))
            print(j, SearchNID(i.ID, 'T', j) ,end=' ')


        writer.write
        print('ID：' + i.ID)
        print('課程：' + i.name)
        print('老師：', end='')
        
        
        print('\n')



# 測試區

In [0]:
id = 70653

url = ('https://ilearn2.fcu.edu.tw/user/index.php?id=' + str(id))
page = s.get(url)
bs_page = BeautifulSoup(page.text, 'html.parser')

# contextid
contextid = bs_page.find_all('body')[0]['class'][11][8:]

# perpage（課程參加人數）
perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]


# url Fix
url = ('https://ilearn2.fcu.edu.tw/user/index.php?contextid='+ contextid +'&id=' + str(id) + '&perpage=' + perpage)

In [0]:
name = '游景盛'

data = {
    'unified-filters[]': name,
    'unified-filter-submitted': '1'
}

# 檢查是否有此人
page = s.post(url, data = data)
bs_page = BeautifulSoup(page.text, 'html.parser')
check = bs_page.find_all(attrs={"id": "user-index-participants-70653_r0_c0"})[0].find('img')['alt'][0:3]

perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

if name == check and perpage == '1':
    print(check + ' 確認')
else:
    print('查無此人')

In [0]:
Teacher_ID = 'T'

while True:
    if len(Teacher_ID) == 6:
        print(name, Teacher_ID)
        break
    else:
        for i in range(0,10):
            
            data = {
                'unified-filters[]': Teacher_ID + str(i),
                'unified-filter-submitted': '1'
            }

            print(Teacher_ID + str(i))

            try:
                page = s.post(url, data = data)
                bs_page = BeautifulSoup(page.text, 'html.parser')
                check = bs_page.find_all(attrs={"id": "user-index-participants-70653_r0_c0"})[0].find('img')['alt'][0:3]

                perpage = bs_page.find_all(attrs={"class": "userlist"})[0].find('p').text[5:]

                print('參加人數：'+ perpage, check, i)

                if name == check and int(perpage) >= 1:
                    Teacher_ID += str(i)
                    print(check + ' 確認\n')
                    if len(Teacher_ID) == 6:
                        break
                else:
                    print('查無此人')
                    print()
            
            except:
                    print()

            print('-----------')
            print()